# Checkpoint 1

**Author:** chenghao Kang (with help of chatGPT)

**Date:** 23/9/2025(finish in one week) 

**Purpose:** To do the EDA for the assginment 2

## Overview of the Notebook

This notebook explores the step activity data of three assigned participants (**IDs: 1844505072, 1927972279, 2022484408**).  
It examines daily, hourly, and minute-level datasets to:  
- Check data completeness and consistency across different levels.  
- Apply adjustments and cleaning for reliable analysis.  
- Validate data alignment (**daily vs hourly**, **hourly vs minute**, **daily vs active minutes**).  
- Prepare for answering the driving question: whether participants achieve 15 minutes of intense activity at different times of the day. 

## Step 1: Load the data
### Purpose:
To import the daily, hourly, and minute-level step datasets, and restrict the analysis to the three participants assigned (**IDs: 1844505072, 1927972279, 2022484408**).  
The goal is to prepare the raw data for further adjustments, cleaning, and correlation analysis.  

**Files loaded:**  
- dailySteps_merged.csv  
- hourlySteps_merged.csv  
- minuteStepsWide_merged.csv  


We begin by loading the three datasets: **Daily**, **Hourly**, and **Minute-level steps**.  


In [185]:
import pandas as pd
import numpy as np
daily = pd.read_csv("dailySteps_merged.csv")
hourly = pd.read_csv("hourlySteps_merged.csv")
minute = pd.read_csv("minuteStepsWide_merged.csv")

ids = [1844505072, 1927972279, 2022484408]
daily = daily[daily["Id"].isin(ids)]
hourly = hourly[hourly["Id"].isin(ids)]
minute = minute[minute["Id"].isin(ids)]


For each participant, I calculated descriptive statistics at both the daily and minute levels to better understand their physical activity patterns.

**Daily-level summary provides:**
    
1.Number of days with available data

2.Average daily steps

3.Maximum and minimum daily steps

4.Standard deviation of daily steps

5.Extra observation: Proportion of active days (days with >0 steps) relative to total days

In [188]:
for pid in ids:
    df_daily = daily[daily['Id'] == pid]
    print(f"\n=== Participant {pid} ===")
    print(f"Number of days: {df_daily.shape[0]}")
    print(f"Average daily steps: {df_daily['StepTotal'].mean():.2f}")
    print(f"Max daily steps: {df_daily['StepTotal'].max()}")
    print(f"Min daily steps: {df_daily['StepTotal'].min()}")
    print(f"Standard deviation: {df_daily['StepTotal'].std():.2f}")

    # Extra observation: Proportion of active days 
    active_days = (df_daily['StepTotal'] > 0).sum()
    prop_active_days = active_days / df_daily.shape[0]
    print(f"Proportion of active days: {prop_active_days:.2%}")



=== Participant 1844505072 ===
Number of days: 31
Average daily steps: 2580.06
Max daily steps: 8054
Min daily steps: 0
Standard deviation: 2712.84
Proportion of active days: 67.74%

=== Participant 1927972279 ===
Number of days: 31
Average daily steps: 916.13
Max daily steps: 3790
Min daily steps: 0
Standard deviation: 1205.27
Proportion of active days: 54.84%

=== Participant 2022484408 ===
Number of days: 31
Average daily steps: 11370.65
Max daily steps: 18387
Min daily steps: 3292
Standard deviation: 2806.99
Proportion of active days: 100.00%


**Minute-level summary provides:**
    
1.Number of non-zero minutes (minutes with >0 steps)

2.Number of missing data points

3.Average steps per minute

4.Maximum and minimum steps in a single minute

5.Minutes with intense activity (defined as >60 steps per minute)

6.Extra observation: Longest streak of consecutive non-zero minutes (indicating sustained activity)

In [190]:
for pid in ids:
    df_minute = minute[minute['Id'] == pid]  # 注意 Id 大小写
    if df_minute.empty:
        print(f"\n=== Participant {pid} ===")
        print("No minute-level data available.")
        continue

    step_cols = [c for c in df_minute.columns if c.startswith('Steps')]
    print(f"\n=== Participant {pid} ===")

    non_zero_minutes = (df_minute[step_cols] > 0).sum().sum()
    missing_data = df_minute[step_cols].isna().sum().sum()
    avg_steps_per_min = df_minute[step_cols].mean().mean()
    max_steps = df_minute[step_cols].max().max()
    min_steps = df_minute[step_cols].min().min()
    intense_minutes = (df_minute[step_cols] > 60).sum().sum()

    print(f"Number of non-zero minutes: {non_zero_minutes}")
    print(f"Missing data points: {missing_data}")
    print(f"Average steps per minute: {avg_steps_per_min:.2f}")
    print(f"Max steps in a minute: {max_steps}")
    print(f"Min steps in a minute: {min_steps}")
    print(f"Minutes with intense activity (>60 steps): {intense_minutes}")

    # Extra observation: Longest streak of consecutive non-zero minutes
    non_zero_flags = (df_minute[step_cols] > 0).astype(int).values.flatten()
    longest_streak = 0
    current = 0
    for val in non_zero_flags:
        if val == 1:
            current += 1
            longest_streak = max(longest_streak, current)
        else:
            current = 0
    print(f"Longest streak of consecutive active minutes: {longest_streak}")



=== Participant 1844505072 ===
Number of non-zero minutes: 3284
Missing data points: 0
Average steps per minute: 1.69
Max steps in a minute: 115
Min steps in a minute: 0
Minutes with intense activity (>60 steps): 137
Longest streak of consecutive active minutes: 32

=== Participant 1927972279 ===
Number of non-zero minutes: 1206
Missing data points: 0
Average steps per minute: 0.63
Max steps in a minute: 117
Min steps in a minute: 0
Minutes with intense activity (>60 steps): 85
Longest streak of consecutive active minutes: 25

=== Participant 2022484408 ===
Number of non-zero minutes: 9217
Missing data points: 0
Average steps per minute: 7.84
Max steps in a minute: 176
Min steps in a minute: 0
Minutes with intense activity (>60 steps): 1856
Longest streak of consecutive active minutes: 64


# Step 2: Apply adjustments

In this step, I made adjustments to ensure consistency across datasets:

Converted date/time fields into proper datetime objects for **Daily**, **Hourly**, and **Minute-level steps**.  
Standardized all column names to lowercase to avoid mismatches.
Checked the time ranges of each dataset to confirm alignment before further analysis.

In [193]:
# Convert date/time fields into proper datetime objects
daily["ActivityDay"] = pd.to_datetime(daily["ActivityDay"], format="%m/%d/%Y")
hourly["ActivityHour"] = pd.to_datetime(hourly["ActivityHour"], format="%m/%d/%Y %I:%M:%S %p")
minute["ActivityHour"] = pd.to_datetime(minute["ActivityHour"], format="%m/%d/%Y %I:%M:%S %p")

# Standardize all column names to lowercase (avoid mismatches)
daily.columns = daily.columns.str.lower()
hourly.columns = hourly.columns.str.lower()
minute.columns = minute.columns.str.lower()

# Check the available time range for each dataset
print("Daily:", daily["activityday"].min(), "to", daily["activityday"].max())
print("Hourly:", hourly["activityhour"].min(), "to", hourly["activityhour"].max())
print("Minute:", minute["activityhour"].min(), "to", minute["activityhour"].max())

Daily: 2016-04-12 00:00:00 to 2016-05-12 00:00:00
Hourly: 2016-04-12 00:00:00 to 2016-05-12 15:00:00
Minute: 2016-04-13 00:00:00 to 2016-05-13 08:00:00


This confirms that the three datasets cover similar but not identical ranges, which is important to keep in mind for later correlation analysis.

# Step 3: Data cleaning

In this step, I focused on ensuring the quality and reliability of the dataset before analysis:

1.Checked for missing values in daily, hourly, and minute datasets.

2.Removed duplicate rows to avoid over-counting.

3.Verified that there are no negative step counts, since step values must logically be non-negative.

In [197]:
print("Daily missing:\n", daily.isnull().sum())
print("Hourly missing:\n", hourly.isnull().sum())
print("Minute missing:\n", minute.isnull().sum())

daily = daily.drop_duplicates()
hourly = hourly.drop_duplicates()
minute = minute.drop_duplicates()

print("Daily negative steps:", (daily["steptotal"] < 0).sum())
print("Hourly negative steps:", (hourly["steptotal"] < 0).sum())

step_cols = [col for col in minute.columns if col.startswith("steps")]
print("Minute negative steps:", (minute[step_cols] < 0).sum().sum())


Daily missing:
 id             0
activityday    0
steptotal      0
dtype: int64
Hourly missing:
 id              0
activityhour    0
steptotal       0
dtype: int64
Minute missing:
 id              0
activityhour    0
steps00         0
steps01         0
steps02         0
               ..
steps55         0
steps56         0
steps57         0
steps58         0
steps59         0
Length: 62, dtype: int64
Daily negative steps: 0
Hourly negative steps: 0
Minute negative steps: 0


The results show that:

There are **no missing values** in any dataset.
    
Duplicate records were removed (if any).

There are **no negative step counts**, confirming the dataset is valid for further analysis.

# Step 4: assumptions

To guide the exploration, we make the following initial assumptions:  

- **Daily Steps Dataset:** Participants are expected to achieve around **10,000 steps per day**, consistent with a common health guideline.  
- **Hourly Steps Dataset:** Step counts should peak during typical active hours (morning commute, lunchtime, and early evening) and be close to zero during sleeping hours (midnight to early morning).  
- **Minute-Level Steps Dataset:** Most minutes will record **zero steps**, with activity occurring in **short bursts** rather than evenly spread throughout the hour.  

**Assumption 1 (Daily dataset completeness):**  
The average daily steps help us check if the assumption of ~10,000 steps is realistic.    


In [202]:
# Aggregate hourly data into daily totals
hourly_day = (
    hourly
    .assign(activityday=hourly['activityhour'].dt.floor('D'))   # convert hour timestamp to day
    .groupby(['id', 'activityday'], as_index=False)['steptotal']
    .sum()
    .rename(columns={'steptotal': 'hourly_sum'})  # rename for clarity
)

# Merge daily dataset with aggregated hourly dataset
daily_compare = (
    daily[['id', 'activityday', 'steptotal']]
    .merge(hourly_day, on=['id', 'activityday'], how='inner')
)

# Compare daily vs hourly totals
daily_compare['diff'] = daily_compare['steptotal'] - daily_compare['hourly_sum'] 
corr1 = daily_compare[['steptotal', 'hourly_sum']].corr().iloc[0,1]           
rmse1 = float(np.sqrt((daily_compare['diff']**2).mean()))                      

print(f"[4.1] Daily vs Hourly-sum: Pearson r = {corr1:.4f}, RMSE = {rmse1:.2f}, n = {len(daily_compare)}")
print(daily_compare.head(10))


[4.1] Daily vs Hourly-sum: Pearson r = 0.9999, RMSE = 80.75, n = 93
           id activityday  steptotal  hourly_sum  diff
0  1844505072  2016-04-12       6697        6697     0
1  1844505072  2016-04-13       4929        4929     0
2  1844505072  2016-04-14       7937        7905    32
3  1844505072  2016-04-15       3844        3844     0
4  1844505072  2016-04-16       3414        3414     0
5  1844505072  2016-04-17       4525        4525     0
6  1844505072  2016-04-18       4597        4597     0
7  1844505072  2016-04-19        197         197     0
8  1844505072  2016-04-20          8           8     0
9  1844505072  2016-04-21       8054        8054     0


**Assumption 2 (Hourly dataset accuracy)**  
This helps confirm whether participants are more active during the day and inactive at night.   


In [204]:
# Select all step-related columns from minute dataset
step_cols = [c for c in minute.columns if c.startswith('steps')]

minute_long = minute.melt(
    id_vars=['id', 'activityhour'],
    value_vars=step_cols,
    var_name='minute_col',
    value_name='steps'
)
# Convert step values to numeric and fill missing values with 0
minute_long['steps'] = pd.to_numeric(minute_long['steps'], errors='coerce').fillna(0)

# Aggregate minute-level steps into hourly totals
minute_to_hour = (
    minute_long
    .groupby(['id', 'activityhour'], as_index=False)['steps']
    .sum()
    .rename(columns={'steps': 'minute_sum'})
)

# Merge hourly dataset with aggregated minute-level dataset
hour_compare = (
    hourly[['id', 'activityhour', 'steptotal']]
    .merge(minute_to_hour, on=['id', 'activityhour'], how='inner')
)

# Compare hourly vs minute totals
hour_compare['diff'] = hour_compare['steptotal'] - hour_compare['minute_sum']
corr2 = hour_compare[['steptotal', 'minute_sum']].corr().iloc[0,1]   
rmse2 = float(np.sqrt((hour_compare['diff']**2).mean()))            

print(f"[4.2] Hourly vs Minute-sum: Pearson r = {corr2:.4f}, RMSE = {rmse2:.2f}, n = {len(hour_compare)}")
print(hour_compare.head(10))


[4.2] Hourly vs Minute-sum: Pearson r = 1.0000, RMSE = 0.00, n = 2131
           id        activityhour  steptotal  minute_sum  diff
0  1844505072 2016-04-13 00:00:00          0           0     0
1  1844505072 2016-04-13 01:00:00          0           0     0
2  1844505072 2016-04-13 02:00:00          0           0     0
3  1844505072 2016-04-13 03:00:00          0           0     0
4  1844505072 2016-04-13 04:00:00          0           0     0
5  1844505072 2016-04-13 05:00:00          0           0     0
6  1844505072 2016-04-13 06:00:00          0           0     0
7  1844505072 2016-04-13 07:00:00          0           0     0
8  1844505072 2016-04-13 08:00:00          0           0     0
9  1844505072 2016-04-13 09:00:00          0           0     0


**Assumption 3 (Minute-level detail)**  
We expect a high proportion of zero-step minutes, but the active minutes show intensity.    


In [206]:
# Add a new column "date" (day-level) from minute timestamps
minute_long['date'] = minute_long['activityhour'].dt.floor('D')

# Mark whether each minute is active (steps > 0)
minute_long['active'] = minute_long['steps'] > 0
# Count active minutes per day (per participant)
active_minutes = (
    minute_long
    .groupby(['id', 'date'], as_index=False)['active']
    .sum()
    .rename(columns={'active': 'active_minutes'})   # rename for clarity
)
# Merge daily total steps with active minutes
daily_active = (
    daily[['id', 'activityday', 'steptotal']]
    .merge(active_minutes, left_on=['id', 'activityday'], right_on=['id', 'date'], how='inner')
    .drop(columns=['date'])   # drop duplicate date column
)

corr3 = daily_active[['steptotal', 'active_minutes']].corr().iloc[0,1]

print(f"[4.3] Daily total vs Active minutes: Pearson r = {corr3:.4f}, n = {len(daily_active)}")
print(daily_active.head(10))



[4.3] Daily total vs Active minutes: Pearson r = 0.9503, n = 90
           id activityday  steptotal  active_minutes
0  1844505072  2016-04-13       4929             248
1  1844505072  2016-04-14       7937             373
2  1844505072  2016-04-15       3844             176
3  1844505072  2016-04-16       3414             147
4  1844505072  2016-04-17       4525             209
5  1844505072  2016-04-18       4597             229
6  1844505072  2016-04-19        197              10
7  1844505072  2016-04-20          8               1
8  1844505072  2016-04-21       8054             323
9  1844505072  2016-04-22       5372             220


- **Daily vs Hourly sum**: Pearson r ≈ 0.9999, RMSE ≈ 80  
- **Hourly vs Minute sum**: Pearson r = 1.0000, RMSE = 0  
- **Daily total vs Active minutes**: Pearson r ≈ 0.95  

These results validate that the data across daily, hourly, and minute levels is consistent. This consistency is important because it allows us to later explore the driving problem — whether individuals achieve 15 minutes of intense activity across different times of day.

## Final Statement
Through this analysis, I learned how to apply a literate programming approach to organize data processing in a clear and logical way. By following the steps of loading data, applying adjustments, cleaning, and correlation analysis, I was able to transform raw wearable device data into meaningful insights. I also gained hands-on experience in handling real-world issues such as missing values, data alignment, and inconsistent data types across multiple datasets.
The three assumptions we made directly connect to our driving problem: “Do participants achieve 15 minutes of intense activity across different times of day (morning, afternoon, evening)?”


### Assumption 1 (Daily dataset completeness): 
We assumed that the daily dataset reflects all participant activity. This ensures that when we analyze step totals, we can trust that the measures capture overall daily activity without large gaps.

### Assumption 2 (Hourly dataset accuracy): 
We assumed that hourly activity is accurately recorded and aligned to the correct day. This is essential when splitting activity into morning, afternoon, and evening, as any misalignment could bias the analysis.

### Assumption 3 (Minute-level detail): 
We assumed that the minute-level dataset correctly represents fine-grained intensity. This matters because the definition of “intense activity” relies on minute-level thresholds (>60 steps per minute).



Together, these assumptions ensure that our data pipeline supports a valid answer to the driving problem. In summary, I learned both technical skills for cleaning and analyzing multi-granularity datasets and critical thinking skills for connecting assumptions to research questions.